In [1]:
# USAGE
# python train_network.py --dataset images --model santa_not_santa.model

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from pyimagesearch.lenet import LeNet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

/home/jorgher/.virtualenvs/cv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# initialize the number of epochs to train for, initia learning rate,
# and batch size
EPOCHS = 15
INIT_LR = 1e-3
BS = 32

# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("imagcars")))

[INFO] loading images...


In [3]:
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (28, 28))
	image = img_to_array(image)
	data.append(image)

	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	label = 1 if label == "cars" else 0
	labels.append(label)

In [4]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [5]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

In [6]:
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [7]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [8]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [9]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/15
23/23 [==============================] - 2s 108ms/step - loss: 0.6131 - acc: 0.6644 - val_loss: 0.4840 - val_acc: 0.7683
Epoch 2/15
23/23 [==============================] - 2s 89ms/step - loss: 0.5110 - acc: 0.7704 - val_loss: 0.4722 - val_acc: 0.8008
Epoch 3/15
23/23 [==============================] - 2s 91ms/step - loss: 0.3993 - acc: 0.8274 - val_loss: 0.3173 - val_acc: 0.8943
Epoch 4/15
23/23 [==============================] - 2s 98ms/step - loss: 0.3918 - acc: 0.8247 - val_loss: 0.2900 - val_acc: 0.8902
Epoch 5/15
23/23 [==============================] - 2s 91ms/step - loss: 0.3274 - acc: 0.8628 - val_loss: 0.2570 - val_acc: 0.8984
Epoch 6/15
23/23 [==============================] - 2s 90ms/step - loss: 0.3320 - acc: 0.8682 - val_loss: 0.2619 - val_acc: 0.9187
Epoch 7/15
23/23 [==============================] - 2s 88ms/step - loss: 0.3008 - acc: 0.8709 - val_loss: 0.2196 - val_acc: 0.9268
Epoch 8/15
23/23 [==============================] - 2s 

In [10]:
# save the model to disk
print("[INFO] serializing network...")
model.save("car-notcar.model")

[INFO] serializing network...


In [11]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Car/Not Car")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("car1.png")
